# Cryptocurrency trading bot.

## Install if needed.

In [1]:
##%pip install python-binance==0.7.5
#%pip install --upgrade python-binance
#%pip install paramiko

## API key management.

In [2]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
client

## Get all available pairs for information needed to trade.

In [3]:
from cryptocurrency.exchange import Cryptocurrency_exchange
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info
exchange_info

,symbol,base_asset,base_asset_precision,quote_asset,quote_precision,quote_asset_precision,base_asset_commission,quote_commission_precision,allow_trailing_stop,cancel_replace_allowed,min_price,max_price,tick_size,step_size,multiplier_up,multiplier_down
0,ETHBTC,ETH,8,BTC,8,8,8,8,True,True,1.000000e-06,922327.0,1.000000e-06,0.0001,5.0,0.2
1,LTCBTC,LTC,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0010,5.0,0.2
2,BNBBTC,BNB,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0010,5.0,0.2
3,NEOBTC,NEO,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0100,5.0,0.2
4,QTUMETH,QTUM,8,ETH,8,8,8,8,True,True,1.000000e-06,1000.0,1.000000e-06,0.1000,5.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410,VITEBUSD,VITE,8,BUSD,8,8,8,8,True,True,1.000000e-05,1000.0,1.000000e-05,0.1000,5.0,0.2
1411,HOOKBTC,HOOK,8,BTC,8,8,8,8,True,True,1.000000e-08,1.0,1.000000e-08,0.1000,5.0,0.2
1412,HOOKUSDT,HOOK,8,USDT,8,8,8,8,True,True,1.000000e-04,1000.0,1.000000e-04,0.1000,5.0,0.2
1413,HOOKBUSD,HOOK,8,BUSD,8,8,8,8,True,True,1.000000e-04,1000.0,1.000000e-04,0.1000,5.0,0.2


## Get conversion table for price conversions.

In [4]:
from_asset = 'BDOT'
to_asset = 'BETH'

from cryptocurrency.conversion import get_shortest_pair_path_between_assets

shortest_pair_path_between_assets = \
    get_shortest_pair_path_between_assets(from_asset, to_asset, 
                                          exchange_info, 
                                          priority='fees')

shortest_pair_path_between_assets

[('BDOT', 'DOT'), ('DOT', 'BUSD'), ('BETH', 'BUSD')]

In [5]:
from cryptocurrency.conversion_table import get_conversion_table
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
conversion_table

,symbol,price_change,price_change_percent,weighted_average_price,close_shifted,close,last_volume,bid_price,bid_volume,ask_price,...,rolling_quote_volume,open_time,first_ID,last_ID,count,base_asset,quote_asset,rolling_base_quote_volume,bid_ask_percent_change,bid_ask_volume_percent_change
close_time,,,,,,,,,,,,,,,,,,,,,
2022-12-08 14:02:18.438,BUSDVAI,0.00000000,0.000,1.03184008,1.033000,1.031000,1163.90000000,1.033000,44.60000,1.034000,...,4.497172e+03,1670439738438,602521,602532,12,BUSD,VAI,4.361951e+03,0.096712,0.899847
2022-12-08 18:35:41.482,SLPBIDR,0.60,1.511,40.35,39.700000,40.300000,3078.00000000,40.000000,140917.00000,40.300000,...,8.410722e+08,1670456141482,1100895,1101447,553,SLP,BIDR,2.087028e+07,0.744417,94.792746
2022-12-08 18:41:29.891,GHSTUSDT,0.00400000,0.392,1.02135044,1.020000,1.024000,2852.30000000,1.023000,1529.60000,1.024000,...,2.496437e+05,1670456489891,2311468,2312880,1413,GHST,USDT,2.437927e+05,0.097656,97.006596
2022-12-08 18:53:29.106,DIABUSD,0.00300000,0.915,0.32576638,0.328000,0.331000,178.20000000,0.330000,4674.90000,0.331000,...,9.160127e+04,1670457209106,4354603,4355288,686,DIA,BUSD,2.767410e+05,0.302115,95.517234
2022-12-08 18:56:59.513,OXTBUSD,0.00220000,2.699,0.08292863,0.081900,0.083700,1690.00000000,0.083400,1671.00000,0.083700,...,4.520531e+04,1670457419513,804418,804871,454,OXT,BUSD,5.400873e+05,0.358423,26.298395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-08 19:07:40.053,OCEANBTC,0.00000027,3.371,0.00000832,0.000008,0.000008,41.00000000,0.000008,27182.00000,0.000008,...,1.455250e+01,1670458060053,8591789,8594489,2701,OCEAN,BTC,1.757548e+06,0.240964,51.901780
2022-12-08 19:07:40.054,OCEANUSDT,0.00780000,5.782,0.14150895,0.134900,0.142700,162.00000000,0.142700,3845.00000,0.142800,...,3.644086e+06,1670458060054,28583485,28604227,20743,OCEAN,USDT,2.553669e+07,0.070028,62.591568
2022-12-08 19:07:40.054,BTCAUD,392.69000000,1.569,25112.88634177,25017.650000,25421.300000,0.00086000,25416.840000,0.02587,25429.280000,...,1.325230e+07,1670458060054,12892125,12917987,25863,BTC,AUD,5.213071e+02,0.048920,96.278377


## Pre-screen buyable base assets in pairs.

In [6]:
from cryptocurrency.conversion_table import get_new_tickers
tickers_list = get_new_tickers(conversion_table=conversion_table)
len(tickers_list)

1415

## Locally (saving time) convert price from an asset to another using existing pairs.

In [7]:
from cryptocurrency.trade import select_asset_with_biggest_wallet
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, 
                                     conversion_table=conversion_table, 
                                     exchange_info=exchange_info, 
                                     as_pair=True)
from_asset, converted_quantity, quantity

('BTC', '39.85', 0.0023129)

## Trade.

In [8]:
import pandas as pd

def get_conversion_table_2(conversion_table_file_name='crypto_logs/conversion_table.txt'):
    return pd.read_csv(conversion_table_file_name, index_col=0)

conversion_table_file_name = 'crypto_logs/conversion_table.txt'
#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True, dump_raw=True)
#conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
#conversion_table

In [9]:
to_asset = 'BTC'
priority = 'fees'

from cryptocurrency.trade import trade
import pandas as pd

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                exchange_info=exchange_info, priority=priority)
from_asset = to_asset
request

[]
Error: Can't trade asset with itself!
Ignoring...


## Main loop buying screened assets in pairs which are connected to the wallet holdings.

In [10]:
import os
os.getcwd()

'/home/samuel/workspace/crypto_logger'

In [11]:
priority = 'fees'
sell_asset = 'BUSD'
take_profit = None
stop_loss = None
bought_asset = None
asset_to_delete = None
crypto_input_log_screened_15s = '~/workspace/crypto_logs/crypto_input_log_15s_screened.txt'
crypto_output_log_screened_15s = '~/workspace/crypto_logs/crypto_output_log_15s_screened.txt'
crypto_output_log_screened_1min = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'
crypto_output_log_screened_30min = '~/workspace/crypto_logs/crypto_output_log_30min_screened.txt'
crypto_output_log_screened_1h = '~/workspace/crypto_logs/crypto_output_log_1h_screened.txt'
crypto_output_log_screened_1d = '~/workspace/crypto_logs/crypto_output_log_1d_screened.txt'
crypto_input_log_screened = crypto_input_log_screened_15s
crypto_output_log_screened = crypto_output_log_screened_1d

keys_file = 'contabo_keys.txt'
with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

#ssh.connect('154.12.239.24', username='sam', port=22, password=password)
#ssh.connect('154.53.55.232', username='sam', port=22, password=password)
ssh.connect('178.238.230.58', username='sam', port=22, password=password)

def get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened):
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
    return pd.read_csv(StringIO(str(ssh_stdout.read().decode('utf-8'))))

def choose_to_asset(from_asset, to_asset, latest_asset, crypto_output_log_screened=crypto_output_log_screened):
    tradable_pairs = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        latest_asset = tradable_assets[-1]
        if from_asset not in tradable_assets:
            to_asset = latest_asset
    return latest_asset, to_asset

def trade_conditionally(client, exchange_info, to_asset, price_when_bought, asset_to_delete):
    conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
    from_asset, converted_quantity, quantity = \
        select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                         exchange_info=exchange_info)
    request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                    exchange_info=exchange_info, priority=priority)
    if request is not None:
        if to_asset != sell_asset:
            pair_bought = request['symbol']
            price_when_bought = float(request['fills'][0]['price'])
        else:
            asset_to_delete = from_asset
        from_asset = to_asset
    return from_asset, to_asset, price_when_bought, asset_to_delete

latest_asset = from_asset
price_when_bought = quantity
while True:
    latest_asset, to_asset = choose_to_asset(from_asset, to_asset, latest_asset, 
                                             crypto_output_log_screened=crypto_output_log_screened)
    if from_asset != to_asset:
        from_asset, to_asset, price_when_bought, asset_to_delete = \
            trade_conditionally(client=client, exchange_info=exchange_info, to_asset=to_asset, 
                                price_when_bought=price_when_bought, asset_to_delete=asset_to_delete)
    time.sleep(2)

ssh.close()

ModuleNotFoundError: No module named 'paramiko'

In [ ]:
priority = 'fees'
sell_asset = 'BUSD'
take_profit = 10.0
stop_loss = None
bought_asset = None
asset_to_delete = None
crypto_input_log_screened_15s = '~/workspace/crypto_logs/crypto_input_log_15s_screened.txt'
crypto_output_log_screened_15s = '~/workspace/crypto_logs/crypto_output_log_15s_screened.txt'
crypto_output_log_screened_1min = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'
crypto_output_log_screened_30min = '~/workspace/crypto_logs/crypto_output_log_30min_screened.txt'
crypto_output_log_screened_1h = '~/workspace/crypto_logs/crypto_output_log_1h_screened.txt'
crypto_output_log_screened_1d = '~/workspace/crypto_logs/crypto_output_log_1d_screened.txt'
crypto_input_log_screened = crypto_input_log_screened_15s
crypto_output_log_screened = crypto_output_log_screened_1d

keys_file = 'contabo_keys.txt'
with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.conversion import get_base_asset_from_pair
from cryptocurrency.exchange import Cryptocurrency_exchange

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

def get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened):
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
    return pd.read_csv(StringIO(str(ssh_stdout.read().decode('utf-8'))))

def choose_to_asset(from_asset, to_asset, latest_asset, crypto_output_log_screened=crypto_output_log_screened):
    tradable_pairs = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        latest_asset = tradable_assets[-1]
        if from_asset not in tradable_assets:
            to_asset = latest_asset
    return latest_asset, to_asset

def trade_conditionally(client, exchange_info, to_asset, price_when_bought, asset_to_delete):
    conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
    from_asset, converted_quantity, quantity = \
        select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                         exchange_info=exchange_info)
    request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                    exchange_info=exchange_info, priority=priority)
    if request is not None:
        if to_asset != sell_asset:
            pair_bought = request['symbol']
            price_when_bought = float(request['fills'][0]['price'])
        else:
            asset_to_delete = from_asset
        from_asset = to_asset
    return from_asset, to_asset, price_when_bought, asset_to_delete

latest_asset = from_asset
price_when_bought = quantity
while True:
    try:
        if asset_to_delete is None:
            latest_asset, to_asset = choose_to_asset(from_asset, to_asset, latest_asset, 
                                                     crypto_output_log_screened=crypto_output_log_screened)
            if from_asset != to_asset:
                from_asset, to_asset, price_when_bought, asset_to_delete = \
                    trade_conditionally(client=client, exchange_info=exchange_info, to_asset=to_asset, 
                                        price_when_bought=price_when_bought, asset_to_delete=asset_to_delete)
            elif bought_asset is not None:
                #conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                price_now = conversion_table[conversion_table['symbol'] == bought_asset]['close'].iat[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                else:
                    asset_to_delete = None
            time.sleep(2)
        else:
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[input_pairs['base_asset'] != asset_to_delete]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = None
    except (IndexError, pd.errors.EmptyDataError):
        time.sleep(0.5)
        continue

ssh.close()

In [ ]:
crypto_input_log_screened_15s = 'crypto_logs/crypto_input_log_15s_screened.txt'
crypto_output_log_screened_15s = 'crypto_logs/crypto_output_log_15s_screened.txt'
crypto_output_log_screened_1min = 'crypto_logs/crypto_output_log_1min_screened.txt'
crypto_output_log_screened_30min = 'crypto_logs/crypto_output_log_30min_screened.txt'
crypto_output_log_screened_1h = 'crypto_logs/crypto_output_log_1h_screened.txt'
crypto_output_log_screened_1d = 'crypto_logs/crypto_output_log_1d_screened.txt'
crypto_input_log_screened = crypto_input_log_screened_15s
crypto_output_log_screened = crypto_output_log_screened_1d

keys_file = 'contabo_keys.txt'
with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

def get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened_1d):
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
    return pd.read_csv(StringIO(str(ssh_stdout.read().decode('utf-8'))))

while True:
    df = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened)

    if df.shape[0] < 1:
        print('None')
    else:
        print(df)

    time.sleep(2)

ssh.close()